## Natural Language Inference

Zero-shot classification can also be done using [Natural Language Inference (NLI)](https://nlpprogress.com/english/natural_language_inference.html), which refers to the task of determining, given a "premise" and a "hypothesis", whether the hypothesis is:

- True (**entailment**)
- False (**contradiction**)
- Undetermined (**neutral**)

Example:

| Premise | Hypothesis | Label |
| --- | --- | --- |
| A soccer game with multiple males playing. | Some men are playing a sport. | entailment |
| A man inspects the uniform of a figure in some East Asian country. | The man is sleeping. | contradiction |
| An older and younger man smiling. | Two men are smiling and laughing at the cats playing on the floor. | neutral |

NLI (Natural Language Inference) can pull off zero-shot classification by turning the task into a true/false question. Here's how:
1. take the text you want to classify (e.g., a movie review) and call it the "premise."
2. craft a "hypothesis" like, “This is a positive review.”
3. The model checks if this hypothesis follows from the premise (entailment = true) or contradicts it (false).
    - If it "entails," label it positive;
    - if it "contradicts," it’s negative.

You don't even need specific training for this.

Zero, single and few-shot classification seem to be an emergent feature of large language models. This feature seems to come about around model sizes of +100M parameters. The effectiveness of a model at a zero, single or few-shot task seems to scale with model size, meaning that larger models (models with more trainable parameters or layers) generally do better at this task.

- Approaches used for NLI include earlier symbolic and statistical approaches to more recent deep learning approaches.
- Benchmark datasets used for NLI include [SNLI](https://paperswithcode.com/dataset/snli), [MultiNLI](https://paperswithcode.com/dataset/multinli), [SciTail](https://paperswithcode.com/dataset/scitail), among others.
- You can get hands-on practice on the SNLI task by following this [d2l.ai chapter](https://d2l.ai/chapter_natural-language-processing-applications/natural-language-inference-and-dataset.html).

Let's grab a [NLI model from HuggingFace](https://huggingface.co/models?pipeline_tag=zero-shot-classification) and demonstrate how to use it for zero-shot classification:

In [ ]:
%pip install -qU datasets transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from transformers import pipeline

# Pre-trained MNLI model
pipe = pipeline(
    model="facebook/bart-large-mnli",
    device='cuda',
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
predictions = pipe("I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)
predictions

{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.5227577686309814,
  0.45814040303230286,
  0.014264780096709728,
  0.0026850062422454357,
  0.0021520727314054966]}

Let's make it multi-label classification via `multi_labels=True`:

In [ ]:
predictions = pipe("I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
    multi_label=True
)
predictions

{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.9987171292304993,
  0.9945850968360901,
  0.18989649415016174,
  0.0007674150983802974,
  0.00038260893779806793]}

Let's try running this on the `rotten_tomatoes` dataset (movie reviews):

In [ ]:
import pandas as pd
from datasets import load_dataset

tomatoes = load_dataset("rotten_tomatoes")

# Pandas for easier control
# tomatoes_train_df = pd.DataFrame(tomatoes["train"])
tomatoes_eval_df = pd.DataFrame(tomatoes["test"])

It takes 44s to classify all 1,066 examples on a run on T4 GPU:

In [ ]:
# Candidate labels
candidate_labels = [
    "very negative movie review",
    "very positive movie review",
]
candidate_labels_dict = {k: v for k, v in enumerate(candidate_labels)}

# Create predictions
predictions = pipe(tomatoes_eval_df.text.tolist(), candidate_labels)

In [ ]:
predictions

{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.9987171292304993,
  0.9945850968360901,
  0.18989649415016174,
  0.0007674150983802974,
  0.00038260893779806793]}

## Exercise

Identify the user intent using NLI:

- `"Hello, I want to get me a Laptop how much does it cost?"`
    - `"BUY Laptop"`
- `"I am very frustrated with your service, and I wanna cancel right now!"`
    - `"CANCEL Subscription"`
- `"Here I bought this Keyboard, but it is not working. I want to get my money back!"`
    - `"REFUND Transaction"`

In [ ]:
# YOUR CODE HERE